In [3]:
import sys
sys.path.append('/content/drive/Othercomputers/내 iMac/Meta_Learning/코스웍/2022 고급기계학습주제 (김광인)/과제/CW1')

In [4]:
from utils.utils import *

In [5]:
import numpy as np
import pandas as pd

from sklearn import svm, metrics

from sklearn.model_selection import GridSearchCV, ParameterGrid, cross_validate
from sklearn.metrics import accuracy_score

In [12]:
# seed fixation
seed = 1
np.random.seed(seed)

In [7]:
# path for colab only
import os
colab = '/content/drive/Othercomputers/내 iMac/Meta_Learning/코스웍/2022 고급기계학습주제 (김광인)/과제/CW1/data'
PATH1 = os.path.join(colab, 'cifar-10-batches-py/data_batch_1')
PATH2 = os.path.join(colab, 'cifar-100-python/train')

# Load data
cifar10 = unpickle(PATH1)
cifar100 = unpickle(PATH2)

# Generate dataset
cifar10 = dataset_generator(cifar10, 3000, cifar_type='cifar10')
cifar100 = dataset_generator(cifar100, 3000, cifar_type='cifar100')

# test path
PATH1_t = os.path.join(colab, 'cifar-10-batches-py/data_batch_2')
PATH2_t = os.path.join(colab, 'cifar-100-python/test')

# Load test data
test_cifar10 = unpickle(PATH1_t)
test_cifar100 = unpickle(PATH2_t)

# Generate test dataset
test_cifar10 = dataset_generator(test_cifar10, 300, cifar_type='cifar10')
test_cifar100 = dataset_generator(test_cifar100, 300, cifar_type='cifar100')

# Normalise
cifar10['data'] = cifar10['data']/255
cifar100['data'] = cifar100['data']/255

test_cifar10['data'] = test_cifar10['data']/255
test_cifar100['data'] = test_cifar100['data']/255

In [8]:
X10 = cifar10['data']
y10 = np.array(cifar10['labels'])
X100 = cifar100['data']
y100 = np.array(cifar100['labels'])

test_X10 = test_cifar10['data']
test_y10 = np.array(test_cifar10['labels'])
test_X100 = test_cifar100['data']
test_y100 = np.array(test_cifar100['labels'])

## Hyperparameter Search: Train/Test Set

In [11]:
param_grid = [
  {'C': [0.01, 0.1, 1, 10, 100], 'kernel': ['linear']},
  {'C': [0.01, 0.1, 1, 10, 100], 'gamma': [0.01, 0.001, 0.0001], 'kernel': ['rbf']},
 ]
 
grid = list(ParameterGrid(param_grid))
grid

[{'C': 0.01, 'kernel': 'linear'},
 {'C': 0.1, 'kernel': 'linear'},
 {'C': 1, 'kernel': 'linear'},
 {'C': 10, 'kernel': 'linear'},
 {'C': 100, 'kernel': 'linear'},
 {'C': 0.01, 'gamma': 0.01, 'kernel': 'rbf'},
 {'C': 0.01, 'gamma': 0.001, 'kernel': 'rbf'},
 {'C': 0.01, 'gamma': 0.0001, 'kernel': 'rbf'},
 {'C': 0.1, 'gamma': 0.01, 'kernel': 'rbf'},
 {'C': 0.1, 'gamma': 0.001, 'kernel': 'rbf'},
 {'C': 0.1, 'gamma': 0.0001, 'kernel': 'rbf'},
 {'C': 1, 'gamma': 0.01, 'kernel': 'rbf'},
 {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'},
 {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'},
 {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'},
 {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'},
 {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'},
 {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'},
 {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'},
 {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}]

### cifar10

In [13]:
import time

train_time = []
inf_time = []
acc = []

for i, args in enumerate(grid):
    print(args)
    clf = svm.SVC(**args, random_state=seed)
    # check time
    start = time.time()
    
    print(f'{i}th classifier training...', end='')
    clf.fit(X10, y10)
    
    end1 = time.time()
    train_time.append(end1-start)
    print( 'time: ', round(end1-start,1), end='')

    y_pred = clf.predict(test_X10)
    ac = accuracy_score(test_y10, y_pred)
    print(' acc: ', ac)

    end2 = time.time()
    inf_time.append(end2-end1)
    acc.append(ac)

# Rounding
tt = [round(t,1) for t in train_time]
it = [round(t,4) for t in inf_time]
ac = [round(a,3) for a in acc]

trial = {'train_time': tt, 'inference_time': it, 'accuracy': ac}

print()
for args, a in zip(grid, ac):
    print(args, a)

print()
print(trial)

{'C': 0.01, 'kernel': 'linear'}
0th classifier training...time:  12.7 acc:  0.41
{'C': 0.1, 'kernel': 'linear'}
1th classifier training...time:  14.2 acc:  0.3466666666666667
{'C': 1, 'kernel': 'linear'}
2th classifier training...time:  15.0 acc:  0.32
{'C': 10, 'kernel': 'linear'}
3th classifier training...time:  14.6 acc:  0.31666666666666665
{'C': 100, 'kernel': 'linear'}
4th classifier training...time:  14.9 acc:  0.31666666666666665
{'C': 0.01, 'gamma': 0.01, 'kernel': 'rbf'}
5th classifier training...time:  18.1 acc:  0.12333333333333334
{'C': 0.01, 'gamma': 0.001, 'kernel': 'rbf'}
6th classifier training...time:  18.3 acc:  0.12333333333333334
{'C': 0.01, 'gamma': 0.0001, 'kernel': 'rbf'}
7th classifier training...time:  19.2 acc:  0.12333333333333334
{'C': 0.1, 'gamma': 0.01, 'kernel': 'rbf'}
8th classifier training...time:  24.7 acc:  0.32
{'C': 0.1, 'gamma': 0.001, 'kernel': 'rbf'}
9th classifier training...time:  17.8 acc:  0.26666666666666666
{'C': 0.1, 'gamma': 0.0001, 'ke

### cifar100

In [ ]:
 import time

train_time = []
inf_time = []
acc = []

for i, args in enumerate(grid):
    print(args)
    clf = svm.SVC(**args, random_state=seed)
    # check time
    start = time.time()
    
    print(f'{i}th classifier training...', end='')
    clf.fit(X100, y100)
    
    end1 = time.time()
    train_time.append(end1-start)
    print( 'time: ', round(end1-start,1), end='')

    y_pred = clf.predict(test_X100)
    ac = accuracy_score(test_y100, y_pred)
    print(' acc: ', ac)

    end2 = time.time()
    inf_time.append(end2-end1)
    acc.append(ac)

# Rounding
tt = [round(t,1) for t in train_time]
it = [round(t,4) for t in inf_time]
ac = [round(a,3) for a in acc]

trial = {'train_time': tt, 'inference_time': it, 'accuracy': ac}

print()
for args, a in zip(grid, ac):
    print(args, a)

print()
print(trial)

{'C': 0.01, 'kernel': 'linear'}
0th classifier training...time:  14.8 acc:  0.14333333333333334
{'C': 0.1, 'kernel': 'linear'}
1th classifier training...time:  17.1 acc:  0.13333333333333333
{'C': 1, 'kernel': 'linear'}
2th classifier training...time:  16.2 acc:  0.12666666666666668
{'C': 10, 'kernel': 'linear'}
3th classifier training...time:  16.2 acc:  0.12666666666666668
{'C': 100, 'kernel': 'linear'}
4th classifier training...time:  16.6 acc:  0.12666666666666668
{'C': 0.01, 'gamma': 0.01, 'kernel': 'rbf'}
5th classifier training...time:  19.8 acc:  0.02
{'C': 0.01, 'gamma': 0.001, 'kernel': 'rbf'}
6th classifier training...time:  19.1 acc:  0.02
{'C': 0.01, 'gamma': 0.0001, 'kernel': 'rbf'}
7th classifier training...time:  18.5 acc:  0.02
{'C': 0.1, 'gamma': 0.01, 'kernel': 'rbf'}
8th classifier training...time:  19.9 acc:  0.02
{'C': 0.1, 'gamma': 0.001, 'kernel': 'rbf'}
9th classifier training...time:  18.7 acc:  0.02666666666666667
{'C': 0.1, 'gamma': 0.0001, 'kernel': 'rbf'}


### Hyperparameter Search: Cross-validation

In [ ]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

In [ ]:
import time
from copy import deepcopy

_time = []
acc = {'mean_acc':[], 'std_acc':[]}

best_acc = -1

for i, args in enumerate(grid):
    print(f'{i}th classifier training...')
    clf = svm.SVC(**args, random_state=seed)
    
    sub_acc = []
    start = time.time()
    for train_index, test_index in skf.split(X10, y10):
        X_train, X_test = X10[train_index], X10[test_index]
        y_train, y_test = y10[train_index], y10[test_index]

        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        
        ac = accuracy_score(y_test, y_pred)
        sub_acc.append(ac)

    end = time.time()
    
    print('time: ', round(end-start,1),end='')
    _time.append(end-start)
    acc['mean_acc'].append(np.mean(sub_acc))
    acc['std_acc'].append(np.std(sub_acc))
    print(' acc: ', acc['mean_acc'][i], end='')
    print(' std: ', acc['std_acc'][i])
    
    # best model
    if np.mean(sub_acc) > best_acc:
        best_acc = np.mean(sub_acc)
        best_clf = deepcopy(clf)

# Rounding
tt = [round(t,1) for t in _time]
ac1 = [round(a,3) for a in acc['mean_acc']]
ac2 = [round(a,3) for a in acc['std_acc']]

trial_cv = {'time': tt, 'mean_accuracy': ac1, 'std_accuracy': ac2}

print()
print(trial_cv)

### cifar100

In [ ]:
import time
from copy import deepcopy

_time = []
acc = {'mean_acc':[], 'std_acc':[]}

best_acc = -1

for i, args in enumerate(grid):
    print(f'{i}th classifier training...')
    clf = svm.SVC(**args, random_state=seed)
    
    sub_acc = []
    start = time.time()
    for train_index, test_index in skf.split(X100, y100):
        X_train, X_test = X100[train_index], X100[test_index]
        y_train, y_test = y100[train_index], y100[test_index]

        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        
        ac = accuracy_score(y_test, y_pred)
        sub_acc.append(ac)

    end = time.time()
    
    print('time: ', round(end-start,1),end='')
    _time.append(end-start)
    acc['mean_acc'].append(np.mean(sub_acc))
    acc['std_acc'].append(np.std(sub_acc))
    print(' acc: ', acc['mean_acc'][i], end='')
    print(' std: ', acc['std_acc'][i])
    
    # best model
    if np.mean(sub_acc) > best_acc:
        best_acc = np.mean(sub_acc)
        best_clf = deepcopy(clf)

# Rounding
tt = [round(t,1) for t in _time]
ac1 = [round(a,3) for a in acc['mean_acc']]
ac2 = [round(a,3) for a in acc['std_acc']]

trial_cv_ = {'time': tt, 'mean_accuracy': ac1, 'std_accuracy': ac2}

print()
print(trial_cv_)

### Varying training set

cifar10 best model: , accuracy: <br>
cifar100 best model: , accuracy:

In [ ]:
cifar10_1000 = dataset_generator(cifar10, 1000, 'cifar10')
cifar10_2000 = dataset_generator(cifar10, 2000, 'cifar10')

In [ ]:
clf = svm.SVC(kernel='rbf', C=10, gamma=0.001, random_state=seed)
clf.fit(cifar10_1000['data'], cifar10_1000['labels'])
y_pred = clf.predict(cifar10_test['data'])
accuracy_score(cifar10_test['labels'], y_pred)

0.36666666666666664

In [ ]:
clf = svm.SVC(kernel='rbf', C=10, gamma=0.001, random_state=seed)
clf.fit(cifar10_2000['data'], cifar10_2000['labels'])
y_pred = clf.predict(cifar10_test['data'])
accuracy_score(cifar10_test['labels'], y_pred)

0.3933333333333333

In [ ]:
cifar100_1000 = dataset_generator(cifar100, 1000, 'cifar100')
cifar100_2000 = dataset_generator(cifar100, 2000, 'cifar100')

In [ ]:
clf = svm.SVC(kernel='rbf', C=10, gamma=0.001, random_state=seed)
clf.fit(cifar100_1000['data'], cifar100_1000['labels'])
y_pred = clf.predict(cifar100_test['data'])
accuracy_score(cifar100_test['labels'], y_pred)

0.10666666666666667

In [ ]:
clf = svm.SVC(kernel='rbf', C=10, gamma=0.001, random_state=seed)
clf.fit(cifar100_2000['data'], cifar100_2000['labels'])
y_pred = clf.predict(cifar100_test['data'])
accuracy_score(cifar100_test['labels'], y_pred)

0.14